In [ ]:
%matplotlib widget

import ipywidgets
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import ndimage
import requests
import gzip
import os
import hashlib

plt.ioff();

# Características no lineales

Incluso utilizando una maquinaria lineal, podemos obtener predictores no lineales a partir del extractor de características.

## Regresión

Recordemos que en los problemas de regresión, a partir de datos de entrenamiento, un algoritmo de aprendizaje produce un predictor que asocia nuevas entradas a nuevas salidas.

La primer decisión de diseño corresponde a responder ¿cuáles son los posibles predictores que puede considerar el algoritmo de aprendizaje? es decir, debemos determinar nuestra **clase de hipótesis**.

Para predictores lineales, la clase de hipótesis es el conjunto de predictores que asocian algúna entrada $x$ al producto punto entre un vector de pesos $\mathbf{w}$ y un vector de características $\phi(x)$.

Si definimos nuestro extractor de características como $\phi(x)=[1, x]$, entonces podemos construir un predictor lineal a partir de pesos $\mathbf{w} = [w_1, w_2]$ donde $w_2$ es la pendiente y $w_1$ la ordenada al origen.

**¿Qué pasa si tenemos datos más complejos?**

In [ ]:
plt.close()

fig = plt.figure()
ax = fig.add_subplot()
np.random.seed(42)
xs = np.random.rand(20) * 5
ys = -0.2*(xs ** 2) + xs + 2 + (np.random.rand(20)-0.5)
ax.scatter(xs, ys, c = "green")
ax.set_xlim((0,5))
ax.set_ylim((0,4.1))
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")

fig.canvas.header_visible = False
display(fig.canvas);

**¿Cómo podemos ajustar un predictor no lineal?**

Una ruta es recurrir inmediatamente a maquinarias más expresivas como redes neuronales.

Pero veamos primero que tanto jugo podemos sacarle a la maquinaria lineal de los predictores.

### Predictores cuadráticos

La clave es observar que el extractor de características $\phi$ puede ser arbitrario.

Podemos definirlo para incluir un término cuadrático.

$$\phi(x) = [1, x, x^2]$$

Incorporando los pesos apropiadamente, podemos definir un predictor no lineal (específicamente, un predictor **cuadrático**).

In [ ]:
def features_squared(x):
    return [1, x, x ** 2]

In [ ]:
def predict(w, x):
    return np.array(w).dot(features_squared(x))

Consideremos como ejemplos tres predictores:

$$\begin{aligned}
\mathbf{w}_1 &= [2, 1, -0.2] \\
\mathbf{w}_2 &= [4, -1, 0.1] \\
\mathbf{w}_3 &= [1, 1, 0]
\end{aligned}$$

La clase de hipótesis es entonces:

$$\mathcal{F} = \left\{ f_\mathbf{w}(x) = \mathbf{w}\cdot\phi(x) \mid \mathbf{w}\in\mathbb{R}^3 \right\}$$

In [ ]:
plt.close()

fig = plt.figure()
ax = fig.add_subplot()
np.random.seed(42)
xs = np.random.rand(20) * 5
ys = -0.2*(xs ** 2) + xs + 2 + (np.random.rand(20)-0.5)
ax.scatter(xs, ys, c = "green")

w1 = [2, 1, -0.2]
w2 = [4, -1, 0.1]
w3 = [1, 1, 0]

xs = np.linspace(0, 5, 100)
ys1 = [predict(w1, x) for x in xs]
ys2 = [predict(w2, x) for x in xs]
ys3 = [predict(w3, x) for x in xs]

ax.plot(xs, ys1, c = "red", label = "$\\mathbf{w}_1$")
ax.plot(xs, ys2, c = "purple", label = "$\\mathbf{w}_2$")
ax.plot(xs, ys3, c = "orange", label = "$\\mathbf{w}_3$")

ax.set_xlim((0,5))
ax.set_ylim((0,4.1))
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.legend()

fig.canvas.header_visible = False
display(fig.canvas);

Si un vector tiene en su última componente el valor de $0$, entonces el predictor asociado es lineal. La clase de hipótesis que consideramos en este ejemplo es un superconjunto de la clase de hipótesis de predictores lineales.

**¿Qué problema puede haber si consideramos entradas con más dimensiones?**

En lugar de $x\in\mathbb{R}$ piensa en $x\in\mathbb{R}^d$ para $d$ grande.

### Predictores por partes constantes

Los predictores cuadráticos siguen siendo algo restrictivos, solo pueden subir y luego bajar suavemente (o viceversa).

Otro tipo de extractor de características divide el espacio de entrada en regiones y permite que el valor predecido de cada región varíe independientemente, produciendo predictores por partes constantes.

$$\phi(x) = [\mathbf{1}[0<x\leq1], \mathbf{1}[1<x\leq2], \mathbf{1}[2<x\leq3], \mathbf{1}[3<x\leq4], \mathbf{1}[4<x\leq5]]$$

Cada componente del vector de características corresponde a una región (por ejemplo $[0,1)$) y es $1$ si $x$ cae dentro de la región y es $0$ en otro caso.

Suponiendo que las regiones son disjuntas, el peso asociado a cada componente/región es precisamente el valor predecido.

Conforme las regiones se consideran más pequeñas, habremos de considerar más características, y la expresividad de la clase de hipótesis crece linealmente. En el límite, puedes esencialmente capturar cualquier predictor que desees.

Considera la clase de hipótesis:

$$\mathcal{F} = \left\{ f_\mathbf{w}(x) = \mathbf{w}\cdot\phi(x) \mid \mathbf{w}\in\mathbb{R}^5 \right\}$$

Y los predictores:

$$\begin{aligned}
\mathbf{w}_1 &= [1, 2, 4, 4, 3] \\
\mathbf{w}_2 &= [4, 3, 3, 2, 1.5]
\end{aligned}$$

In [ ]:
def features_piecewise(x):
    return [
        1 if 0 < x <= 1 else 0,
        1 if 1 < x <= 2 else 0,
        1 if 2 < x <= 3 else 0,
        1 if 3 < x <= 4 else 0,
        1 if 4 < x <= 5 else 0,
    ]

In [ ]:
def predict(w, x):
    return np.array(w).dot(features_piecewise(x))

In [ ]:
plt.close()

fig = plt.figure()
ax = fig.add_subplot()

w1 = [1, 2, 4, 4, 2]
w2 = [4, 3, 3, 2, 1.5]

xs = np.linspace(0, 5, 500)
ys1 = [predict(w1, x) for x in xs]
ys2 = [predict(w2, x) for x in xs]

ax.plot(xs, ys1, c = "red", label = "$\\mathbf{w}_1$")
ax.plot(xs, ys2, c = "purple", label = "$\\mathbf{w}_2$")

ax.set_xlim((0,5))
ax.set_ylim((0,4.1))
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.legend()

fig.canvas.header_visible = False
display(fig.canvas);

**¿Qué pasa si $x$ no es un escalar si no un vector $d$-dimensional?**

Considera que cada componente deberá asociarse a una de $B$ regiones.

### Predictores con estructura periódica

> Quítale lo aburrido, ponle lo divertido

**¡Incorpora las características que quieras!**

$$\begin{aligned}
\phi(x) &= [1, x, x^2, \cos(3x)] \\
\mathbf{w}_1 &= [1, 1, -0.1, 1] \\
\mathbf{w}_2 &= [3, -1, 0.1, 0.5]
\end{aligned}$$

In [ ]:
def features_periodic(x):
    return [1, x, x ** 2, np.cos(3*x)]

In [ ]:
def predict(w, x):
    return np.array(w).dot(features_periodic(x))

In [ ]:
plt.close()

fig = plt.figure()
ax = fig.add_subplot()

w1 = [1, 1, -0.1, 1]
w2 = [3, -1, 0.1, 0.5]

xs = np.linspace(0, 5, 100)
ys1 = [predict(w1, x) for x in xs]
ys2 = [predict(w2, x) for x in xs]

ax.plot(xs, ys1, c = "red", label = "$\\mathbf{w}_1$")
ax.plot(xs, ys2, c = "purple", label = "$\\mathbf{w}_2$")

ax.set_xlim((0,5))
ax.set_ylim((0,4.1))
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.legend()

fig.canvas.header_visible = False
display(fig.canvas);

¿Cómo podemos obtener predictores no lineales si seguimos utilizando la maquinaria de los predictores lineales?

Estamos usando el término *lineal* de manera ambigüa.

- La respuesta $\mathbf{w}\cdot\phi(x)$ es lineal con respecto a $\mathbf{w}$
- La respuesta $\mathbf{w}\cdot\phi(x)$ es lineal con respecto a $\phi(x)$
- La respuesta $\mathbf{w}\cdot\phi(x)$ **no** es lineal con respecto a $x$

Es posible que esta no linealidad con respecto a $x$ ni siquiera tenga sentido ¡$x$ no necesariamente es un vector! pudiera ser una imágen, un archivo PDF, una novela representada como cadena de caracteres...

Desde la perspectiva del algoritmo de aprendizaje, el cuál observa a $\phi(x)$ pero no a $x$), la linealidad nos permite optimizar los pesos eficientemente.

Si la respuesta es lineal con respecto a $\mathbf{w}$ y la función de pérdida es convexa (lo cuál es el caso para la pérdida cuadrática, de articulación y la logística, pero no es el caso para la pérdida cero-uno), entonces minimizar la pérdida de entrenamiento es un problema de **optimización convexa** y el descenso de gradiente (con un tamaño de paso apropiado) está destinado a converger al mínimo global.

## Clasificación

En problemas de clasificación también puedes definir características arbitrarias para producir clasificadores no lineales.

Recuerda que en la clasificación binaria el clasificador regresa el signo de la respuesta.

El clasificador puede entonces ser representado por su frontera de decisión, la cuál divide el espacio de entrada en dos regiones: los puntos con respuesta positiva y los puntos con respuesta negativa.

### Clasificadores cuadráticos

Consideremos como ejemplos un clasificador con frontera de decisión no lineal, partiendo el espacio de entrada con un círculo: los puntos dentro del círculo son clasificados con $+1$ y los puntos fuera del círculo son clasificados con $-1$.

$$\begin{align}
\phi(x) &= [x_1, x_2, x_1^2 + x_2^2] \\
f_\mathbf{w}(x) &= \text{sign}(\mathbf{w}\cdot\phi(x)) \\
\mathbf{w}_1 &= [2, 2, -1] \\
f_{\mathbf{w}_1}(x) &= \text{sign}([2, 2, -1]\cdot[x_1, x_2, x_1^2 + x_2^2]) \\
&= \text{sign}(2x_1 + 2x_2 - x_1^2 - x_2^2) \\
&= \begin{cases}
+1 &\text{si } 2x_1 + 2x_2 - x_1^2 - x_2^2 \geq 0 \\
-1 &\text{en otro caso}
\end{cases} \\
&= \begin{cases}
+1 &\text{si } x_1^2 + x_2^2 - 2x_1 - 2x_2 + 2 \leq 2 \\
-1 &\text{en otro caso}
\end{cases} \\
&= \begin{cases}
+1 &\text{si } x_1^2 - 2x_1 + 1 + x_2^2 - 2x_2 + 1 \leq 2 \\
-1 &\text{en otro caso}
\end{cases} \\
&= \begin{cases}
+1 &\text{si } (x_1 - 1)^2 + (x_2 - 1)^2 \leq 2 \\
-1 &\text{en otro caso}
\end{cases}
\end{align}$$

In [ ]:
def features_circle(x):
    return [x[0], x[1], x[0]**2 - x[1]**2]

In [ ]:
def predict(w, x):
    return np.sign(np.array(w).dot(features_circle(x)))

In [ ]:
plt.close()

fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot()

w = [2, 2, -1]

xs = list(np.linspace(-3, 3, 100))
x1s, x2s = [], []
for x1 in xs:
    for x2 in xs:
        x1s.append(x1)
        x2s.append(x2)

x1sPos, x2sPos = [], []
x1sNeg, x2sNeg = [], []
for x1, x2 in zip(x1s, x2s):
    if predict(w, [x1, x2]) >= 0:
        x1sPos.append(x1)
        x2sPos.append(x2)
    else:
        x1sNeg.append(x1)
        x2sNeg.append(x2)

ax.scatter(x1sPos, x2sPos, c = "purple", marker = ".", label = "$+1$")
ax.scatter(x1sNeg, x2sNeg, c = "orange", marker = ".", label = "$-1$")

ax.set_xlim((-3, 3))
ax.set_ylim((-3, 3))
ax.set_xlabel("$x_1$")
ax.set_ylabel("$x_2$")
ax.legend()

fig.canvas.header_visible = False
display(fig.canvas);

# Plantillas de características

Ahora discutiremos cómo podemos usar plantillas de características para construir características de manera flexible.

Recuerda que la clase de hipótesis $\mathcal{F}$ es el conjunto de predictores considerados por nuestro algoritmo de aprendizaje.  En el caso de los predictores lineales, $\mathcal{F}$ está dado por alguna función de $\mathbf{w}\cdot\phi(x)$ para todas las $\mathbf{w}$.

El aprendizaje es el proceso de elegir un predictor particular $f_\mathbf{w}$ de $\mathcal{F}$ dado un conjunto de entrenamiento.

La pregunta que nos concierne ahora es ¿Cómo elegimos $\mathcal{F}$? Ya discutimos predictores lineales, cuadráticos, etc. ¿Qué hace sentido para la aplicación que trabajamos?

Si la clase de hipótesis no contiene buenos predictores, no importa que tan bueno sea nuestro aprendizaje.  La pregunta es si la extracción de características es suficientemente buena para **expresar** buenos predictores.

## Validación de correos electrónicos

Considera la tarea de predecir si una cadena es una dirección de correo electrónico válida o no.

Supongamos que el clasificador $f_\mathbf{w}$ es lineal, dado por algún extractor de características $\phi$.

Extraer características apropiadas es todo un arte y requiere intuición sobre el problema en cuestión y también sobre las capacidades de los algoritmos de aprendizaje máquina.

La idea clave es que las características deben representar propiedades de $x$ que **puedan ser** relevantes para predecir $y$.

$$x = \mathtt{abc@gmail.com}$$
$$\Downarrow$$
$$\phi$$
$$\Downarrow$$
| característica | valor |
|----------------|-------|
| `length>10`    | 1     |
| `fracOfAlpha`  | 0.85  |
| `contains_@`   | 1     |
| `endsWith_com` | 1     |
| `endsWith_org` | 0     |

El extractor debe producir un conjunto de pares que consisten del nombre de la característica y su valor. Podemos extraer información sobre la longitud, la proporción de caracteres alfanuméricos, o si contiene algunas subcadenas.

No te preocupes si agregas características que resultan ser irrelevantes, ya que el algoritmo de aprendizaje puede en principio decidir ignorar esa característica, aunque quizá requiera procesar más datos para ello.

Un vector de características formalmente es solo una lista de números, pero hemos aumentado cada característica en el vector con un nombre. El vector de pesos también es solo una lista de números, también  lo aumentaremos con un nombre para cada peso.

$$\text{Vector de pesos } \mathbf{w}\in\mathbb{R}^d$$
|                |       |
|----------------|-------|
| `length>10`    | -1.2  |
| `fracOfAlpha`  | 0.6   |
| `contains_@`   | 3     |
| `endsWith_com` | 2.2   |
| `endsWith_org` | 1.4   |

$$\text{Vector de características } \phi(x)\in\mathbb{R}^d$$
|                |       |
|----------------|-------|
| `length>10`    | 1     |
| `fracOfAlpha`  | 0.85  |
| `contains_@`   | 1     |
| `endsWith_com` | 1     |
| `endsWith_org` | 0     |

Recuerda que la respuesta es simplemente el producto punto entre el vector de pesos y el vector de características. En otras palabras, la respuesta agrega la contribución de cada característica, ponderada apropiadamente.

$$\begin{aligned}
\mathbf{w}\cdot\phi(x) &= \sum_{j=1}^d w_j\phi(x)_j \\
&= -1.2(1) + 0.6(0.85) + 3(1) + 2.2(1) + 1.4(0) \\
&= 4.510
\end{aligned}$$

Cada peso $w_j$ determina cómo el valor de característica correspondiente $\phi(x)_j$ contribuye a la predicción.  Si $w_j$ es positivo, entonces la presencia de una característica $j$ favorece una clasificación positiva (por ejemplo, que termine la dirección en `.com`). En cambio, si $w_j$ es negativa, entonces la presencia de una característica $j$ favorece una clasificación negativa (por ejemplo, que la longitud sea mayor a 10).  La magnitud de $w_j$ mide la fuerza o importancia de su contribución.

In [ ]:
def features(x):
    return {
        "length>10": int(len(x) > 10),
        "fracOfAlpha": len([c for c in x if c.isalpha()]) / len(x),
        "contains_@": int("@" in x),
        "endsWith_com": int(len(x) >= 3 and x[-1:-4:-1][::-1] == "com"),
        "endsWith_org": int(len(x) >= 3 and x[-1:-4:-1][::-1] == "org"),
    }

In [ ]:
features("abc@gmail.com")

En el ejemplo anterior definimos ciertas características que creíamos que serían útiles para detectar direcciones de correo electrónico.  Sin embargo, es fácil omitir algunas características (como por ejemplo `endsWith_mx`), y también pueden haber otras características que permiten una buena predicción pero no son intuitivas.

Necesitamos un principio organizacional, una manera sistemática de definir características...

## Plantillas de características

Una **plantilla de características** es un grupo de características que se calculas de manera similar.

En lugar de definir características individuales como `endsWith_com`, podemos definir una plantilla que se expanda a todas las características que computen la coincidencia de los tres últimos caracteres de $x$.

$$\mathtt{abc@gmail.com}$$
$$\Downarrow$$
$$\text{los últimos tres caracteres son \_\_\_}$$
$$\Downarrow$$
|                |   |
|----------------|---|
| `endsWith_aaa` | 0 |
| `endsWith_aab` | 0 |
| `endsWith_aac` | 0 |
| $\dots$        |   |
| `endsWith_com` | 1 |
| $\dots$        |   |
| `endsWith_zzz` | 0 |

No necesitamos conocer cuáles patrones particulares (como sufijos de tres caracteres) son útiles, solo que la **existencia** de algunos patrones pudieran ser útiles.

Entonces es trabajo del algoritmo de aprendizaje discernir cuáles patrones son útiles asignando pesos apropiadamente.

**¿Qué implementación de vector es más apropiada?** arreglos para características densas, diccionarios para características dispersas.

In [ ]:
def endsWithFeatures(x):
    if len(x) < 3:
        return {}
    suffix = x[-1:-4:-1][::-1]
    if not suffix.isalpha():
        return {}
    return {
        "endsWith_" + suffix: 1
    }

In [ ]:
endsWithFeatures("abc@gmail.com")

In [ ]:
endsWithFeatures("pickles69@420.gglol")

In [ ]:
def features(x):
    return {
        "length>10": int(len(x) > 10),
        "fracOfAlpha": len([c for c in x if c.isalpha()]) / len(x),
        "contains_@": int("@" in x),
    } | endsWithFeatures(x)

In [ ]:
features("abc@gmail.com")

In [ ]:
features("eay2@eacuna.org")

# Transformando imágenes

Juguemos un poco con los datos de MNIST

In [ ]:
def fetch(url):
    hash = hashlib.md5(url.encode("utf-8")).hexdigest()
    path = os.path.join(".", hash)
    if os.path.isfile(path):
        with open(path, "rb") as f:
            data = f.read()
    else:
        with open(path, "wb") as f:
            data = requests.get(url).content
            f.write(data)
    return np.frombuffer(
        gzip.decompress(data),
        dtype=np.uint8,
    ).copy()

In [ ]:
image_data = fetch("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz")
label_data = fetch("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz")

In [ ]:
[image_magic] = image_data[0:4][::-1].copy().view(np.uint32)
[image_total] = image_data[4:8][::-1].copy().view(np.uint32)
[image_rows, image_cols] = image_data[8:16][::-1].copy().view(np.uint32)
images = image_data[16:].reshape((image_total, image_rows, image_cols))
[label_magic] = label_data[0:4][::-1].copy().view(np.uint32)
[label_total] = label_data[4:8][::-1].copy().view(np.uint32)
labels = label_data[8:]

In [ ]:
def show_example(img, digit = None, label = None):
    plt.close()
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.imshow(img, cmap=mpl.cm.gray_r)
    if digit is not None:
        ax.set_title(f"La imagen contiene el dígito {digit}")
    elif label is not None:
        ax.set_title(label)
    display(fig)

In [ ]:
n = 3
show_example(images[n], labels[n])

Esta imagen parece un uno, pero hay muchas formas de escribir este simple numeral, usualmente como un palito, a veces rotado, a veces no, ¿qué tan rotado?

Consideramos de ejemplo esta imagen.

In [ ]:
image = images[n]

## Rotación

Cada pixel en la imágen original es movido a otro de acuerdo a la transformación

$$T = \begin{bmatrix}
\cos\theta & -\sin\theta \\
\sin\theta & \cos\theta
\end{bmatrix}$$

Pensamos en un pixel como una coordenada en el plano y un valor de intensidad luminosa. En particular consideramos a la coordenada como un vector en $\mathbb{R}^2$.

In [ ]:
plt.close()

hmin, hmax = 0, 30
vmin, vmax = 0, 30

x_init, x_min, x_max, x_step = 14, 0, 27, 1
y_init, y_min, y_max, y_step = 12, 0, 27, 1
a_init, a_min, a_max, a_step = 0, -2, 2, 0.01

def rotVec(x, y, theta):
    return [
        np.cos(theta)*x - np.sin(theta)*y,
        np.sin(theta)*x + np.cos(theta)*y,
    ]

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot()
pixel = ax.arrow(0, 0, x_init, y_init, head_width=0.75, color="black")
rot = rotVec(x_init, y_init, np.pi*a_init)
pixelT = ax.arrow(0, 0, rot[0], rot[1], head_width=0.75, color="blue")
ax.imshow(image, cmap=mpl.cm.autumn, alpha=1)

ax.set_xlim((hmin, hmax))
ax.set_ylim((vmin, vmax))
ax.invert_yaxis()

def update_plot(x, y, a):
    pixel.set_data(x = 0, y = 0, dx = x, dy = y)
    rot = rotVec(x, y, np.pi*a)
    pixelT.set_data(x = 0, y = 0, dx = rot[0], dy = rot[1])
    fig.canvas.draw()
    fig.canvas.flush_events()

widget = ipywidgets.interactive(
    update_plot,
    x = ipywidgets.IntSlider(
        orientation="horizontal",
        description="x",
        value=x_init,
        min=x_min,
        max=x_max,
        step=x_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    y = ipywidgets.IntSlider(
        orientation="horizontal",
        description="y",
        value=y_init,
        min=y_min,
        max=y_max,
        step=y_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    a = ipywidgets.FloatSlider(
        orientation="horizontal",
        description="angle (*pi)",
        value=a_init,
        min=a_min,
        max=a_max,
        step=a_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
)

fig.canvas.header_visible = False
display(widget)
display(fig.canvas)

Regresemos a la imagen del $1$ ladeado.

Consideremos de ejemplo el pixel en $(14, 12)$, es decir, en el renglón $12$ y la columna $14$. El cuál en la imagen tiene valor de:

In [ ]:
image[14, 12]

Supongamos que queremos rotar esta imagen $12^\circ$ en contra de las manecillas del reloj, es decir, $-12^\circ$ o lo equivalente en radianes $-12\pi/180 = -0.20943951023931953$.

In [ ]:
theta = -12*np.pi/180

Podemos encontrar la ubicación del pixel en la imagen transformada con el producto entre la matriz de transformación y la posición original.

$$\begin{bmatrix}
\cos\theta & -\sin\theta \\
\sin\theta & \cos\theta
\end{bmatrix}\begin{bmatrix}
14 \\ 12
\end{bmatrix} = \begin{bmatrix}
14\cos\theta - 12\sin\theta \\
14\sin\theta + 12\cos\theta
\end{bmatrix} = \begin{bmatrix}
14\cos\frac{-12\pi}{180} - 12\sin\frac{-12\pi}{180} \\
14\sin\frac{-12\pi}{180} + 12\cos\frac{-12\pi}{180}
\end{bmatrix} = \begin{bmatrix}
16.1890067 \\
8.82700754
\end{bmatrix}$$

In [ ]:
T = np.array([[np.cos(theta), -np.sin(theta)],
              [np.sin(theta), np.cos(theta)]])
pos = np.array([[14],
                [12]])
np.dot(T, pos)

Esta transformación nos puede llevar a posiciones no enteras, por lo que existe un único pixel en esta nueva posición, queda entre varios pixeles.  Vamos a tomar el piso de ambas coordenadas para concluir que la nueva posición de este pixel es $(16, 8)$, es decir, el renglón $8$, columna $16$.

Esta operación de rotación se realiza con respecto al origen $(0, 0)$, podemos generalizar la transformación para realizarla con respecto a cualquier otro punto al incorporar un desplazamiento (offset). Trasladamos todas las posiciones al punto de rotación, rotamos con la matriz de transformación y posteriormente volvemos a trasladar para obtener las posiciones finales.

In [ ]:
plt.close()

hmin, hmax = 0, 30
vmin, vmax = 0, 30

x_init, x_min, x_max, x_step = 14, 0, 27, 1
y_init, y_min, y_max, y_step = 12, 0, 27, 1
a_init, a_min, a_max, a_step = 0, -2, 2, 0.01
cx_init, cx_min, cx_max, cx_step = 0, 0, 27, 1
cy_init, cy_min, cy_max, cy_step = 0, 0, 27, 1

def rotVec(x, y, theta):
    return [
        np.cos(theta)*x - np.sin(theta)*y,
        np.sin(theta)*x + np.cos(theta)*y,
    ]

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot()
pixel = ax.arrow(0, 0, x_init, y_init, head_width=0.75, color="black")
rot = rotVec(x_init, y_init, np.pi*a_init)
pixelT = ax.arrow(0, 0, rot[0], rot[1], head_width=0.75, color="blue")
center = ax.plot([cx_init], [cy_init], color="blue", marker = "o", linestyle = "None")
ax.imshow(image, cmap=mpl.cm.autumn, alpha=1)

ax.set_xlim((hmin, hmax))
ax.set_ylim((vmin, vmax))
ax.invert_yaxis()

def update_plot(x, y, a, cx, cy):
    pixel.set_data(x = 0, y = 0, dx = x, dy = y)
    rot = rotVec(x - cx, y - cy, np.pi*a)
    pixelT.set_data(x = 0, y = 0, dx = rot[0] + cx, dy = rot[1] + cy)
    center[0].set_data([cx], [cy])
    fig.canvas.draw()
    fig.canvas.flush_events()

widget = ipywidgets.interactive(
    update_plot,
    x = ipywidgets.IntSlider(
        orientation="horizontal",
        description="x",
        value=x_init,
        min=x_min,
        max=x_max,
        step=x_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    y = ipywidgets.IntSlider(
        orientation="horizontal",
        description="y",
        value=y_init,
        min=y_min,
        max=y_max,
        step=y_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    a = ipywidgets.FloatSlider(
        orientation="horizontal",
        description="angle (*pi)",
        value=a_init,
        min=a_min,
        max=a_max,
        step=a_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    cx = ipywidgets.IntSlider(
        orientation="horizontal",
        description="cx",
        value=cx_init,
        min=cx_min,
        max=cx_max,
        step=cx_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    cy = ipywidgets.IntSlider(
        orientation="horizontal",
        description="cy",
        value=cy_init,
        min=cy_min,
        max=cy_max,
        step=cy_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
)

fig.canvas.header_visible = False
display(widget)
display(fig.canvas)

La siguiente implementación se encarga de rotar imagenes dado un ángulo y un desplazamiento.

In [ ]:
def rotate(image, angle, offset = [0,0]):
    T = np.array([[np.cos(angle), -np.sin(angle)],
                  [np.sin(angle), np.cos(angle)]])
    h, w = image.shape
    off_x = offset[0]
    off_y = offset[1]
    imageT = np.zeros_like(image)
    for y in range(h):
        for x in range(w):
            value = image[y, x]
            pos = np.array([[x - off_x], [y - off_y]])
            posT = np.dot(T, pos)
            xT = off_x + int(posT[0][0])
            yT = off_y + int(posT[1][0])
            if (0 <= xT < w) and (0 <= yT < h):
                imageT[yT,xT] = value
    return imageT

Las siguientes tres imagenes corresponden respectivamente a:
- La imagen sin rotación
- La imagen rotada $-12^\circ$ con respecto al origen
- La imagen rotada $-12^\circ$ con respecto al centro de la imagen $(14, 14)$

In [ ]:
show_example(image, label="Imagen original")
show_example(rotate(image, theta, [0, 0]), label="Imagen rotada al origen")
show_example(rotate(image, theta, [14, 14]), label="Imagen rotada a (14,14)")

Observemos que aparecen algunos artefactos extraños, algunos pixeles sin tinta, algunas regiones mas delgadas o anchas a comparación de la original.  A este efecto se le conoce como *aliasing*, realizamos la transformación considerando que cada posición de los pixeles se encontraba en $\mathbb{R}^2$, las transformamos linealmente en el continuo y luego discretizamos, perdiendo información y produciendo estos artefactos.

Una antigua técnica para combatir los efectos del aliasing consiste en primero escalar hacia arriba la imagen original dos o cuatro veces, luego realizar la rotación y finalmente escalar hacia abajo promediando los valores en los pixeles.

In [ ]:
def upscale(image, factor = 2):
    h, w = image.shape
    imageT = np.zeros((factor * h, factor * w))
    nh, nw = imageT.shape
    for y in range(nh):
        for x in range(nw):
            value = image[y // factor, x // factor]
            imageT[y, x] = value
    return imageT

In [ ]:
def downscale(image, factor = 2):
    h, w = image.shape
    imageT = np.zeros((h // factor, w // factor))
    nh, nw = imageT.shape
    for y in range(h):
        for x in range(w):
            value = image[y, x] / factor
            imageT[y // factor, x // factor] += value
    return imageT

Las siguientes imagenes muestran respectivamente:
- La imagen sin rotación
- La imagen con rotación sin anti-aliasing
- La imagen con rotación y anti-aliasing 2x
- La imagen con rotación y anti-aliasing 4x
- La imagen con rotación y anti-aliasing 8x

In [ ]:
show_example(image, label="Imagen original")
show_example(rotate(image, theta, [14, 14]), label="Imagen rotada a (14,14)")
show_example(rotate(upscale(image, 2), theta, [2*14, 2*14]), label="Imagen con anti-aliasing 2X rotada a (14,14)")
show_example(rotate(upscale(image, 4), theta, [4*14, 4*14]), label="Imagen con anti-aliasing 4X rotada a (14,14)")
show_example(rotate(upscale(image, 8), theta, [8*14, 8*14]), label="Imagen con anti-aliasing 8X rotada a (14,14)")

In [ ]:
show_example(image, label="Imagen original")
show_example(rotate(image, theta, [14, 14]), label="Imagen rotada")
show_example(downscale(rotate(upscale(image, 2), theta, [2*14, 2*14]), 2), label="Imagen rotada antialiasing 2X")
show_example(downscale(rotate(upscale(image, 4), theta, [4*14, 4*14]), 4), label="Imagen rotada antialiasing 4X")
show_example(downscale(rotate(upscale(image, 8), theta, [8*14, 8*14]), 8), label="Imagen rotada antialiasing 8X")

Podemos crear una abstracción sobre este proceso, a partir de una imagen, una matriz de transformación y un punto de origen para la transformación, realizar la operación utilizando esta técnica anti-aliasing.

In [ ]:
def transform(image, T, offset = [0,0], antialiasing = 2):
    image = upscale(image, antialiasing)
    h, w = image.shape
    off_x = offset[0] * antialiasing
    off_y = offset[1] * antialiasing
    imageT = np.zeros_like(image)
    for y in range(h):
        for x in range(w):
            value = image[y, x]
            pos = np.array([[x - off_x], [y - off_y]])
            posT = np.dot(T, pos)
            xT = int(off_x + int(posT[0][0]))
            yT = int(off_y + int(posT[1][0]))
            if (0 <= xT < w) and (0 <= yT < h):
                imageT[yT,xT] = value
    return downscale(imageT, antialiasing)

La previa operación de rotación se obtiene de la siguiente manera:

In [ ]:
def rotationT(angle):
    return np.array([[np.cos(angle), -np.sin(angle)],
                     [np.sin(angle), np.cos(angle)]])

In [ ]:
show_example(image, label="Imagen original")
show_example(transform(
    image,
    rotationT(theta),
    offset = [14, 14],
), label="Imagen rotada")

In [ ]:
plt.close()

hmin, hmax = -1, 28
vmin, vmax = -1, 28

x_init, x_min, x_max, x_step = 14, 0, 27, 1
y_init, y_min, y_max, y_step = 12, 0, 27, 1
a_init, a_min, a_max, a_step = 0, -2, 2, 0.01
cx_init, cx_min, cx_max, cx_step = 14, 0, 27, 1
cy_init, cy_min, cy_max, cy_step = 14, 0, 27, 1

def rotVec(x, y, theta):
    return [
        np.cos(theta)*x - np.sin(theta)*y,
        np.sin(theta)*x + np.cos(theta)*y,
    ]

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot()
pixel = ax.arrow(0, 0, x_init, y_init, head_width=0.75, color="black")
rot = rotVec(x_init, y_init, np.pi*a_init)
pixelT = ax.arrow(0, 0, rot[0], rot[1], head_width=0.75, color="blue")
center = ax.plot([cx_init], [cy_init], color="blue", marker = "o", linestyle = "None")
ax.imshow(image, cmap=mpl.cm.autumn, alpha=0.5)
imgplot = ax.imshow(image, cmap=mpl.cm.autumn, alpha=0.5)

ax.set_xlim((hmin, hmax))
ax.set_ylim((vmin, vmax))
ax.invert_yaxis()
ax.set_title("Transformación con rotationT")

def update_plot(x, y, a, cx, cy):
    pixel.set_data(x = 0, y = 0, dx = x, dy = y)
    rot = rotVec(x - cx, y - cy, np.pi*a)
    pixelT.set_data(x = 0, y = 0, dx = rot[0] + cx, dy = rot[1] + cy)
    center[0].set_data([cx], [cy])
    imgplot.set(data=transform(
        image,
        rotationT(np.pi*a),
        offset = [cx, cy],
    ), clip_on=False,)
    fig.canvas.draw()
    fig.canvas.flush_events()

widget = ipywidgets.interactive(
    update_plot,
    x = ipywidgets.IntSlider(
        orientation="horizontal",
        description="x",
        value=x_init,
        min=x_min,
        max=x_max,
        step=x_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    y = ipywidgets.IntSlider(
        orientation="horizontal",
        description="y",
        value=y_init,
        min=y_min,
        max=y_max,
        step=y_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    a = ipywidgets.FloatSlider(
        orientation="horizontal",
        description="angle (*pi)",
        value=a_init,
        min=a_min,
        max=a_max,
        step=a_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    cx = ipywidgets.IntSlider(
        orientation="horizontal",
        description="cx",
        value=cx_init,
        min=cx_min,
        max=cx_max,
        step=cx_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    cy = ipywidgets.IntSlider(
        orientation="horizontal",
        description="cy",
        value=cy_init,
        min=cy_min,
        max=cy_max,
        step=cy_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
)

fig.canvas.header_visible = False
display(widget)
display(fig.canvas)

## Estiramiento

In [ ]:
def xstretchT(factor):
    return np.array([[factor, 0],
                     [0, 1]])

In [ ]:
def ystretchT(factor):
    return np.array([[1, 0],
                     [0, factor]])

In [ ]:
show_example(image)
show_example(transform(
    image,
    xstretchT(2),
    offset = [14, 14],
))

In [ ]:
plt.close()

hmin, hmax = -1, 28
vmin, vmax = -1, 28

x_init, x_min, x_max, x_step = 14, 0, 27, 1
y_init, y_min, y_max, y_step = 12, 0, 27, 1
k_init, k_min, k_max, k_step = 1, -3, 3, 0.01
cx_init, cx_min, cx_max, cx_step = 14, 0, 27, 1
cy_init, cy_min, cy_max, cy_step = 14, 0, 27, 1

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot()
pixel = ax.arrow(0, 0, x_init, y_init, head_width=0.75, color="black")
pixelT = ax.arrow(0, 0, x_init, y_init, head_width=0.75, color="blue")
center = ax.plot([cx_init], [cy_init], color="blue", marker = "o", linestyle = "None")
ax.imshow(image, cmap=mpl.cm.autumn, alpha=0.5)
imgplot = ax.imshow(image, cmap=mpl.cm.autumn, alpha=0.5)

ax.set_xlim((hmin, hmax))
ax.set_ylim((vmin, vmax))
ax.invert_yaxis()
ax.set_title("Transformación con xstretchT")

def update_plot(x, y, k, cx, cy):
    pixel.set_data(x = 0, y = 0, dx = x, dy = y)
    T = xstretchT(k)
    pix = T.dot(np.array([x - cx, y - cy]))
    pixelT.set_data(x = 0, y = 0, dx = pix[0] + cx, dy = pix[1] + cy)
    center[0].set_data([cx], [cy])
    imgplot.set(data=transform(
        image,
        T,
        offset = [cx, cy],
    ), clip_on=False,)
    fig.canvas.draw()
    fig.canvas.flush_events()

widget = ipywidgets.interactive(
    update_plot,
    x = ipywidgets.IntSlider(
        orientation="horizontal",
        description="x",
        value=x_init,
        min=x_min,
        max=x_max,
        step=x_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    y = ipywidgets.IntSlider(
        orientation="horizontal",
        description="y",
        value=y_init,
        min=y_min,
        max=y_max,
        step=y_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    k = ipywidgets.FloatSlider(
        orientation="horizontal",
        description="k",
        value=k_init,
        min=k_min,
        max=k_max,
        step=k_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    cx = ipywidgets.IntSlider(
        orientation="horizontal",
        description="cx",
        value=cx_init,
        min=cx_min,
        max=cx_max,
        step=cx_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    cy = ipywidgets.IntSlider(
        orientation="horizontal",
        description="cy",
        value=cy_init,
        min=cy_min,
        max=cy_max,
        step=cy_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
)

fig.canvas.header_visible = False
display(widget)
display(fig.canvas)

In [ ]:
show_example(image)
show_example(transform(
    image,
    ystretchT(0.5),
    offset = [14, 14],
))

In [ ]:
plt.close()

hmin, hmax = -1, 28
vmin, vmax = -1, 28

x_init, x_min, x_max, x_step = 14, 0, 27, 1
y_init, y_min, y_max, y_step = 12, 0, 27, 1
k_init, k_min, k_max, k_step = 1, -3, 3, 0.01
cx_init, cx_min, cx_max, cx_step = 14, 0, 27, 1
cy_init, cy_min, cy_max, cy_step = 14, 0, 27, 1

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot()
pixel = ax.arrow(0, 0, x_init, y_init, head_width=0.75, color="black")
pixelT = ax.arrow(0, 0, x_init, y_init, head_width=0.75, color="blue")
center = ax.plot([cx_init], [cy_init], color="blue", marker = "o", linestyle = "None")
ax.imshow(image, cmap=mpl.cm.autumn, alpha=0.5)
imgplot = ax.imshow(image, cmap=mpl.cm.autumn, alpha=0.5)

ax.set_xlim((hmin, hmax))
ax.set_ylim((vmin, vmax))
ax.invert_yaxis()
ax.set_title("Transformación con ystretchT")

def update_plot(x, y, k, cx, cy):
    pixel.set_data(x = 0, y = 0, dx = x, dy = y)
    T = ystretchT(k)
    pix = T.dot(np.array([x - cx, y - cy]))
    pixelT.set_data(x = 0, y = 0, dx = pix[0] + cx, dy = pix[1] + cy)
    center[0].set_data([cx], [cy])
    imgplot.set(data=transform(
        image,
        T,
        offset = [cx, cy],
    ), clip_on=False,)
    fig.canvas.draw()
    fig.canvas.flush_events()

widget = ipywidgets.interactive(
    update_plot,
    x = ipywidgets.IntSlider(
        orientation="horizontal",
        description="x",
        value=x_init,
        min=x_min,
        max=x_max,
        step=x_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    y = ipywidgets.IntSlider(
        orientation="horizontal",
        description="y",
        value=y_init,
        min=y_min,
        max=y_max,
        step=y_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    k = ipywidgets.FloatSlider(
        orientation="horizontal",
        description="k",
        value=k_init,
        min=k_min,
        max=k_max,
        step=k_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    cx = ipywidgets.IntSlider(
        orientation="horizontal",
        description="cx",
        value=cx_init,
        min=cx_min,
        max=cx_max,
        step=cx_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    cy = ipywidgets.IntSlider(
        orientation="horizontal",
        description="cy",
        value=cy_init,
        min=cy_min,
        max=cy_max,
        step=cy_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
)

fig.canvas.header_visible = False
display(widget)
display(fig.canvas)

In [ ]:
def squeezeT(factor):
    return np.array([[factor, 0],
                     [0, 1 / factor]])

In [ ]:
show_example(image)
show_example(transform(
    image,
    squeezeT(2),
    offset = [14, 14],
))
show_example(transform(
    image,
    squeezeT(0.5),
    offset = [14, 14],
))

In [ ]:
plt.close()

hmin, hmax = -1, 28
vmin, vmax = -1, 28

x_init, x_min, x_max, x_step = 14, 0, 27, 1
y_init, y_min, y_max, y_step = 12, 0, 27, 1
k_init, k_min, k_max, k_step = 1, -3, 3, 0.01
cx_init, cx_min, cx_max, cx_step = 14, 0, 27, 1
cy_init, cy_min, cy_max, cy_step = 14, 0, 27, 1

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot()
pixel = ax.arrow(0, 0, x_init, y_init, head_width=0.75, color="black")
pixelT = ax.arrow(0, 0, x_init, y_init, head_width=0.75, color="blue")
center = ax.plot([cx_init], [cy_init], color="blue", marker = "o", linestyle = "None")
ax.imshow(image, cmap=mpl.cm.autumn, alpha=0.5)
imgplot = ax.imshow(image, cmap=mpl.cm.autumn, alpha=0.5)

ax.set_xlim((hmin, hmax))
ax.set_ylim((vmin, vmax))
ax.invert_yaxis()
ax.set_title("Transformación con squeezeT")

def update_plot(x, y, k, cx, cy):
    pixel.set_data(x = 0, y = 0, dx = x, dy = y)
    T = squeezeT(k)
    pix = T.dot(np.array([x - cx, y - cy]))
    pixelT.set_data(x = 0, y = 0, dx = pix[0] + cx, dy = pix[1] + cy)
    center[0].set_data([cx], [cy])
    imgplot.set(data=transform(
        image,
        T,
        offset = [cx, cy],
    ), clip_on=False,)
    fig.canvas.draw()
    fig.canvas.flush_events()

widget = ipywidgets.interactive(
    update_plot,
    x = ipywidgets.IntSlider(
        orientation="horizontal",
        description="x",
        value=x_init,
        min=x_min,
        max=x_max,
        step=x_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    y = ipywidgets.IntSlider(
        orientation="horizontal",
        description="y",
        value=y_init,
        min=y_min,
        max=y_max,
        step=y_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    k = ipywidgets.FloatSlider(
        orientation="horizontal",
        description="k",
        value=k_init,
        min=k_min,
        max=k_max,
        step=k_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    cx = ipywidgets.IntSlider(
        orientation="horizontal",
        description="cx",
        value=cx_init,
        min=cx_min,
        max=cx_max,
        step=cx_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    cy = ipywidgets.IntSlider(
        orientation="horizontal",
        description="cy",
        value=cy_init,
        min=cy_min,
        max=cy_max,
        step=cy_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
)

fig.canvas.header_visible = False
display(widget)
display(fig.canvas)

## Cizallamiento

In [ ]:
def xshearingT(factor):
    return np.array([[1, factor],
                     [0, 1]])

def yshearingT(factor):
    return np.array([[1, 0],
                     [factor, 1]])

In [ ]:
show_example(image)
show_example(transform(
    image,
    xshearingT(0.5),
    offset = [14, 14],
))
show_example(transform(
    image,
    yshearingT(0.5),
    offset = [14, 14],
))

In [ ]:
plt.close()

hmin, hmax = -1, 28
vmin, vmax = -1, 28

x_init, x_min, x_max, x_step = 14, 0, 27, 1
y_init, y_min, y_max, y_step = 12, 0, 27, 1
k_init, k_min, k_max, k_step = 0, -3, 3, 0.01
cx_init, cx_min, cx_max, cx_step = 14, 0, 27, 1
cy_init, cy_min, cy_max, cy_step = 14, 0, 27, 1

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot()
pixel = ax.arrow(0, 0, x_init, y_init, head_width=0.75, color="black")
pixelT = ax.arrow(0, 0, x_init, y_init, head_width=0.75, color="blue")
center = ax.plot([cx_init], [cy_init], color="blue", marker = "o", linestyle = "None")
ax.imshow(image, cmap=mpl.cm.autumn, alpha=0.5)
imgplot = ax.imshow(image, cmap=mpl.cm.autumn, alpha=0.5)

ax.set_xlim((hmin, hmax))
ax.set_ylim((vmin, vmax))
ax.invert_yaxis()
ax.set_title("Transformación con xshearingT")

def update_plot(x, y, k, cx, cy):
    pixel.set_data(x = 0, y = 0, dx = x, dy = y)
    T = xshearingT(k)
    pix = T.dot(np.array([x - cx, y - cy]))
    pixelT.set_data(x = 0, y = 0, dx = pix[0] + cx, dy = pix[1] + cy)
    center[0].set_data([cx], [cy])
    imgplot.set(data=transform(
        image,
        T,
        offset = [cx, cy],
    ), clip_on=False,)
    fig.canvas.draw()
    fig.canvas.flush_events()

widget = ipywidgets.interactive(
    update_plot,
    x = ipywidgets.IntSlider(
        orientation="horizontal",
        description="x",
        value=x_init,
        min=x_min,
        max=x_max,
        step=x_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    y = ipywidgets.IntSlider(
        orientation="horizontal",
        description="y",
        value=y_init,
        min=y_min,
        max=y_max,
        step=y_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    k = ipywidgets.FloatSlider(
        orientation="horizontal",
        description="k",
        value=k_init,
        min=k_min,
        max=k_max,
        step=k_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    cx = ipywidgets.IntSlider(
        orientation="horizontal",
        description="cx",
        value=cx_init,
        min=cx_min,
        max=cx_max,
        step=cx_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    cy = ipywidgets.IntSlider(
        orientation="horizontal",
        description="cy",
        value=cy_init,
        min=cy_min,
        max=cy_max,
        step=cy_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
)

fig.canvas.header_visible = False
display(widget)
display(fig.canvas)

In [ ]:
plt.close()

hmin, hmax = -1, 28
vmin, vmax = -1, 28

x_init, x_min, x_max, x_step = 14, 0, 27, 1
y_init, y_min, y_max, y_step = 12, 0, 27, 1
k_init, k_min, k_max, k_step = 0, -3, 3, 0.01
cx_init, cx_min, cx_max, cx_step = 14, 0, 27, 1
cy_init, cy_min, cy_max, cy_step = 14, 0, 27, 1

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot()
pixel = ax.arrow(0, 0, x_init, y_init, head_width=0.75, color="black")
pixelT = ax.arrow(0, 0, x_init, y_init, head_width=0.75, color="blue")
center = ax.plot([cx_init], [cy_init], color="blue", marker = "o", linestyle = "None")
ax.imshow(image, cmap=mpl.cm.autumn, alpha=0.5)
imgplot = ax.imshow(image, cmap=mpl.cm.autumn, alpha=0.5)

ax.set_xlim((hmin, hmax))
ax.set_ylim((vmin, vmax))
ax.invert_yaxis()
ax.set_title("Transformación con yshearingT")

def update_plot(x, y, k, cx, cy):
    pixel.set_data(x = 0, y = 0, dx = x, dy = y)
    T = yshearingT(k)
    pix = T.dot(np.array([x - cx, y - cy]))
    pixelT.set_data(x = 0, y = 0, dx = pix[0] + cx, dy = pix[1] + cy)
    center[0].set_data([cx], [cy])
    imgplot.set(data=transform(
        image,
        T,
        offset = [cx, cy],
    ), clip_on=False,)
    fig.canvas.draw()
    fig.canvas.flush_events()

widget = ipywidgets.interactive(
    update_plot,
    x = ipywidgets.IntSlider(
        orientation="horizontal",
        description="x",
        value=x_init,
        min=x_min,
        max=x_max,
        step=x_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    y = ipywidgets.IntSlider(
        orientation="horizontal",
        description="y",
        value=y_init,
        min=y_min,
        max=y_max,
        step=y_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    k = ipywidgets.FloatSlider(
        orientation="horizontal",
        description="k",
        value=k_init,
        min=k_min,
        max=k_max,
        step=k_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    cx = ipywidgets.IntSlider(
        orientation="horizontal",
        description="cx",
        value=cx_init,
        min=cx_min,
        max=cx_max,
        step=cx_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
    cy = ipywidgets.IntSlider(
        orientation="horizontal",
        description="cy",
        value=cy_init,
        min=cy_min,
        max=cy_max,
        step=cy_step,
        layout=ipywidgets.Layout(width="90%"),
    ),
)

fig.canvas.header_visible = False
display(widget)
display(fig.canvas)

Las imagenes usualmente se escriben con un ladeado sobre el eje $x$. La transformación de cizallamiento, en particular `xshearingT` nos puede ayudar a rectificar lo ladeado de las imágenes.  La transformación es:

$$\mathtt{xshearingT}(k) = \begin{bmatrix}
1 & k \\
0 & 1
\end{bmatrix}$$

Por lo que para un pixel en $(x, y)$, esta transformación lo asocia a la posición $(x + ky, y)$. Por lo que un desplazamiento en $x$ será indistinto para el efecto buscado, en cambio, el desplazamiento en $y$ será determinante.

In [ ]:
show_example(transform(
    image,
    xshearingT(0.5),
    offset = [0, 0],
))
show_example(transform(
    image,
    xshearingT(0.5),
    offset = [0, 4.5],
))
show_example(transform(
    image,
    xshearingT(0.5),
    offset = [0, 7],
))
show_example(transform(
    image,
    xshearingT(0.5),
    offset = [0, 11.5],
))
show_example(transform(
    image,
    xshearingT(0.5),
    offset = [0, 14],
))

A partir de una imágen, ¿Cómo podemos determinar el factor de cizallamiento en $x$? ¿Cómo podemos determinar el desplazamiento en $y$ adecuado?

Para el desplazamiento, podemos calcular en qué punto se encuentra el valor central de la imagen sobre el eje $x$. Esto nos permitiría obtener el punto de balance sobre este eje para realizar la transformación allí.

In [ ]:
# Utilizar un meshgrid puede resultar más eficiente que ciclos anidados!
idx_x, idx_y = np.mgrid[:28,:28]
total = np.sum(image)

In [ ]:
mu_x = np.sum(idx_x * image) / total

In [ ]:
mu_x

De manera análoga podemos identificar el valor central sobre $y$.

In [ ]:
mu_y = np.sum(idx_y * image) / total

In [ ]:
mu_y

Para determinar el factor de cizallamiento, podemos utilizar la covarianza para determinar la dependencia lineal entre los valores en $x$ y $y$. La normalizamos por la varianza sobre el eje de las $x$, contemplando la variación en la escala del eje.

In [ ]:
var_x = np.sum((idx_x - mu_x)**2 * image) / total

In [ ]:
cov_xy = np.sum((idx_y - mu_y) * (idx_x - mu_x) * image) / total

In [ ]:
-cov_xy / var_x

Creamos una abstracción sobre estos cálculos para rectificar lo ladeado de las imágenes.

In [ ]:
def unskew_params(image):
    h, w = image.shape
    idx_x, idx_y = np.mgrid[:h, :w]
    total = np.sum(image)
    mu_x = np.sum(idx_x * image) / total
    mu_y = np.sum(idx_y * image) / total
    var_x = np.sum((idx_x - mu_x)**2 * image) / total
    cov_xy = np.sum((idx_y - mu_y) * (idx_x - mu_x) * image) / total
    return -cov_xy / var_x, [mu_x, mu_y]

In [ ]:
np.mgrid[:5,:5][0]

In [ ]:
def plot_drawing():
    img = plt.imread("drawing.png")
    img = (255*(1-img)).astype('uint8')
    k, [cx, cy] = unskew_params(img)
    plt.close()
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.imshow(img, cmap=mpl.cm.magma)
    ax.plot([0, 27], [cy, cy], c="green", lw=3.0)
    ax.arrow(13, 0, -k*7, 0, head_width=0.75, lw=3.0, color="green")
    ax.arrow(13, 27, k*7, 0, head_width=0.75, lw=3.0, color="green")
    ax.set_xlim((0, 27))
    display(fig)

In [ ]:
plot_drawing()

In [ ]:
def unskew_image(image):
    k, offset = unskew_params(image)
    return transform(
        image,
        xshearingT(k),
        offset = offset,
    )

In [ ]:
n = np.random.randint(len(images))
show_example(images[n], labels[n])
show_example(unskew_image(images[n]))

Transformemos las imagenes para rectificar lo ladeadas que están. Esto puede resultar muy tardado, por lo que aprovecharemos el multiprocesamiento de Python.

In [ ]:
from multiprocessing import Pool

In [ ]:
def unskew_images(images):
    with Pool() as pool:
        return pool.map(unskew_image, images)

In [ ]:
%%time

imagesT = unskew_images(images)

Y probemos qué tal funcionan nuestras rústicas maquinarias lineales de predicción.

In [ ]:
def features(x):
    return x

In [ ]:
def zero_weights():
    return np.zeros(image_rows * image_cols)

In [ ]:
def predict(w, x):
    return np.sign(w.dot(features(x)))

In [ ]:
def margin(x, y, w):
    return predict(w, x)*y

In [ ]:
def loss_hinge(x, y, w):
    return max(1 - margin(x, y, w), 0)

In [ ]:
def grad_loss_hinge(x, y, w):
    return -features(x)*y if margin(x, y, w) < 1 else np.zeros_like(w)

In [ ]:
def train_loss(Dtrain, loss, w):
    examples = len(Dtrain)
    total = sum(loss(x, y, w) for x, y in Dtrain)
    return total / examples

In [ ]:
def grad_train_loss(Dtrain, grad_loss, w):
    examples = len(Dtrain)
    total = sum(grad_loss(x, y, w) for x, y in Dtrain)
    return total / examples

In [ ]:
def eta_decreasing(init_eta):
    def decreaser(n):
        return init_eta / np.sqrt(n)
    return decreaser

In [ ]:
def GD(data, loss, grad_loss, init_weights, eta, epochs):
    w_hist = []
    tl_hist = []
    w = init_weights()
    for t in range(1, epochs + 1):
        tl = train_loss(data, loss, w)
        gtl = grad_train_loss(data, grad_loss, w)
        w_hist.append(w)
        tl_hist.append(tl)
        w = w - eta * gtl
    return w, (w_hist, tl_hist)

In [ ]:
DtrainRaw = [
    (images[i].reshape(image_rows * image_cols),
     +1 if labels[i] == 0 else -1)
    for i in range(image_total)
]

In [ ]:
DtrainT = [
    (imagesT[i].reshape(image_rows * image_cols),
     +1 if labels[i] == 0 else -1)
    for i in range(image_total)
]

In [ ]:
%%time

gd1_w, (gd1_ws, gd1_tls) = GD(
            data = DtrainRaw,
            loss = loss_hinge,
       grad_loss = grad_loss_hinge,
    init_weights = zero_weights,
             eta = 0.1,
          epochs = 20,
)

In [ ]:
%%time

gd2_w, (gd2_ws, gd2_tls) = GD(
            data = DtrainT,
            loss = loss_hinge,
       grad_loss = grad_loss_hinge,
    init_weights = zero_weights,
             eta = 0.1,
          epochs = 20,
)

In [ ]:
plt.close()

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot()

epochs = list(range(1, len(gd1_tls) + 1))
ax.plot(epochs, gd1_tls, label = "GD Raw")
ax.plot(epochs, gd2_tls, label = "GD Deskew")
ax.legend()
ax.set_title("Learning with linear classifier and hinge loss")
ax.set_xlabel("epoch")
ax.set_ylabel("TrainLoss")
ax.set_xlim((epochs[0], epochs[-1]))

fig.canvas.header_visible = False
display(fig.canvas);

<img src="https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExMHRiZ21sOWdibDY5cmdidmtwNzRqNmo4NXc4MzRyaHNuZ28zbnhqMyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/tkApIfibjeWt1ufWwj/giphy.gif"></img>

## ¿Qué otros tipos de transformaciones podemos utilizar?

Un kernel o matriz de convolución es una pequeña matriz (como las transformaciones lineales que ya vimos) que nos permite aplicar algunos efectos como blur, sharpening, detección de bordes ,etc.

Estas matrices se aplican diferente a las que ya discutimos. Obtenemos los nuevos valores de los pixeles iterando la matriz sobre la imagen original, la matriz puede tomar valores de una vecindad del pixel destino, permitiendo incorporar información local al pixel en la transformación.

In [ ]:
def convolve(image, T):
    h = image.shape[0]
    w = image.shape[1]
    th, tw = T.shape
    assert th == tw
    assert th % 2 == 1
    half = th // 2
    imageT = np.zeros_like(image)
    for y in range(half, h - half):
        for x in range(half, w - half):
            imageT[y,x] = (T * image[y - half : y + half + 1, x - half : x + half + 1]).sum()
    return imageT

In [ ]:
def identityC():
    return np.array([[0, 0, 0],
                     [0, 1, 0],
                     [0, 0, 0]])

def edges1C():
    return np.array([[0, -1, 0],
                     [-1, 4, -1],
                     [0, -1, 0]])

def edges2C():
    return np.array([[-1, -1, -1],
                     [-1, 8, -1],
                     [-1, -1, -1]])

def sharpenC():
    return np.array([[0, -1, 0],
                     [-1, 5, -1],
                     [0, -1, 0]])

def boxblurC():
    return 1/9 * np.ones((3,3))

def gaussianblur1C():
    return 1/16 * np.array([[1, 2, 1],
                            [2, 4, 2],
                            [1, 2, 1]])

def gaussianblur2C():
    return 1/256 * np.array([[1, 4, 6, 4, 1],
                             [4, 16, 24, 16, 4],
                             [6, 24, 36, 24, 6],
                             [4, 16, 24, 16, 4],
                             [1, 4, 6, 4, 1]])

def unsharpmaskingC():
    return -1/256 * np.array([[1, 4, 6, 4, 1],
                              [4, 16, 24, 16, 4],
                              [6, 24, -476, 24, 6],
                              [4, 16, 24, 16, 4],
                              [1, 4, 6, 4, 1]])

In [ ]:
n = np.random.randint(len(images))
show_example(images[n], labels[n])
show_example(convolve(images[n], edges1C()))
show_example(convolve(images[n], edges2C()))
show_example(convolve(images[n], sharpenC()))
show_example(convolve(images[n], boxblurC()))
show_example(convolve(images[n], gaussianblur1C()))
show_example(convolve(images[n], gaussianblur2C()))
show_example(convolve(images[n], unsharpmaskingC()))

In [ ]:
def random_homie():
    base_path = "../20241/fotos/"
    try:
        photos = os.listdir(base_path)
        photo = np.random.choice(photos)
        return plt.imread(base_path + photo).mean(axis=2)
    except:
        return random_homie()

In [ ]:
def show_homie(homie):
    plt.close()
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.imshow(homie, cmap=mpl.cm.gray)
    display(fig)

In [ ]:
homie = random_homie()
show_homie(homie)
show_homie(convolve(homie, edges1C()))
show_homie(convolve(homie, edges2C()))
show_homie(convolve(homie, sharpenC()))
show_homie(convolve(homie, boxblurC()))
show_homie(convolve(homie, gaussianblur1C()))
show_homie(convolve(homie, gaussianblur2C()))
show_homie(convolve(homie, unsharpmaskingC()))

La convolución de `unsharpmaskingC` se mira chistosa, usémosla junto con `deskew` para entrenar un clasificador lineal y comparar la pérdida de aprendizaje.

In [ ]:
def unsharp_image(image):
    return convolve(image, unsharpmaskingC())

In [ ]:
def unsharp_images(images):
    with Pool() as pool:
        return pool.map(unsharp_image, images)

In [ ]:
%%time

imagesTC = unsharp_images(imagesT)

In [ ]:
DtrainTC = [
    (imagesTC[i].reshape(image_rows * image_cols),
     +1 if labels[i] == 0 else -1)
    for i in range(image_total)
]

In [ ]:
%%time

gd3_w, (gd3_ws, gd3_tls) = GD(
            data = DtrainTC,
            loss = loss_hinge,
       grad_loss = grad_loss_hinge,
    init_weights = zero_weights,
             eta = 0.1,
          epochs = 20,
)

In [ ]:
plt.close()

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot()

epochs = list(range(1, len(gd1_tls) + 1))
ax.plot(epochs, gd1_tls, label = "GD Raw")
ax.plot(epochs, gd2_tls, label = "GD Deskew")
ax.plot(epochs, gd3_tls, label = "GD Deskew Unsharp")
ax.legend()
ax.set_title("Learning with linear classifier and hinge loss")
ax.set_xlabel("epoch")
ax.set_ylabel("TrainLoss")
ax.set_xlim((epochs[0], epochs[-1]))

fig.canvas.header_visible = False
display(fig.canvas);

In [ ]:
gd2_w, (gd2_ws, gd2_tls) = GD(
            data = DtrainT,
            loss = loss_hinge,
       grad_loss = grad_loss_hinge,
    init_weights = zero_weights,
             eta = 0.1,
          epochs = 100,
)
gd3_w, (gd3_ws, gd3_tls) = GD(
            data = DtrainTC,
            loss = loss_hinge,
       grad_loss = grad_loss_hinge,
    init_weights = zero_weights,
             eta = 0.1,
          epochs = 100,
)

In [ ]:
print("TrainLoss con Deskew + Unsharpen:", gd3_tls[-1])
print("TrainLoss con Deskew:", gd2_tls[-1])

In [ ]:
gd3_tls[-1] < gd2_tls[-1]

<img src="https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExeDI1MnY1eWIwM2V3NGphYXNjMWFkOWlwMWxzMms2MzRqNWo4N2tmciZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/pynZagVcYxVUk/giphy.gif"></img>

# Dudas?

Consideremos el predictor para el dígito $0$.

In [ ]:
def learn(Dtrain):
    return GD(
        data = Dtrain,
        loss = loss_hinge,
   grad_loss = grad_loss_hinge,
init_weights = zero_weights,
         eta = 0.1,
      epochs = 100,
    )

In [ ]:
w0, (ws0, tls0) = learn(DtrainRaw)

In [ ]:
train_loss(DtrainRaw, loss_hinge, w0)

Este es un clasificador para el cero, permite predecir si una imagen contiene cero o no contiene cero.

Hemos logrado una pérdida de entrenamiento de $0.19743333333333332$, pero esta baja pérdida es engañosa.
¿Qué nos dice?
¿Esperamos predecir correctamente imagenes con el $0$ un $80\%$ de las veces?

<img src="https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExcG1pOTdjdGxrbWYwYnYzM3lwd3p5ZHRlbWczZGI1dWhzMWo3NGI5eSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/TnDWOEP7dI9LG/giphy.gif"></img>

¿Cuántos dígitos cero predecimos correctamente como cero?

In [ ]:
def correct_prediction0(image, label):
    predicted = np.sign(w0.dot(features(image.reshape(image_rows * image_cols))))
    return label == 0 and predicted > 0

In [ ]:
sum(correct_prediction0(image, label) for image, label in zip(images, labels))

Caracoles, ninguna de las imágenes que tienen escrito un cero son predecidas correctamente...
¿Será que nos falta entrenar con más épocas?
¿Será que debemos replantear el problema de manera distinta?

<img src="https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExbHJsYzFhcmJqZG52dHA4cGk1bDRlM2JzMGQwMmdkczR3amJyajlwbiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/3o7TKSxdQJIoiRXHl6/giphy.gif"></img>

Implementa un clasificador multi-clase para los dígitos 0, 1, 2, 3, 4, 5, 6, 7, 8, 9. Te invito a incorporar características adicionales, utilizar plantillas de características, incorporar distintas transformaciones lineales y convoluciones.

¿Qué tanto puedes bajar la pérdida de entrenamiento?
¿Puedes lograr predecir dígitos?

Fortuna y gloria le esperan a quien se aventure en esta exploración y presente un reporte de sus hallazgos.